In [17]:
import scanpy as sc
import numpy as np
import anndata2ri as ad

In [18]:
# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
ad.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

sc.settings.verbosity = 3
sc.logging.print_versions()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                         9.2.0
anndata2ri                  1.1
asttokens                   NA
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
bottleneck                  1.3.5
cffi                        1.15.1
cloudpickle                 2.2.0
colorama                    0.4.6
cycler                      0.10.0
cython_runtime              NA
cytoolz                     0.12.0
dask                        2022.11.0
dateutil                    2.8.2
debugpy                     1.6.3
decorator                   5.1.1
defusedxml                  0.7.1
entrypoints                 0.4
executing                   1.2.0
fsspec                      2022.11.0
google                      NA
h5py                        3.7.0
hypergeom_ufunc             NA
igraph                      0.10.2
ipykernel           

In [19]:
%%R

# IMPORTANT! Use Seurat v3.0.2 as subsequent versions are packaged with "Reticulate" which interferes with rpy2
library('Seurat') 
library('base')
library('patchwork')

In [20]:
%%R 
setwd('/home/jovyan/conga_example_dataset_v1')

R[write to console]: Error in setwd("/home/jovyan/conga_example_dataset_v1") : 
  cannot change working directory




Error in setwd("/home/jovyan/conga_example_dataset_v1") : 
  cannot change working directory


RInterpreterError: Failed to parse and evaluate line "setwd('/home/jovyan/conga_example_dataset_v1')\n".
R error message: 'Error in setwd("/home/jovyan/conga_example_dataset_v1") : \n  cannot change working directory'

In [21]:
%%R

# Read the data from the HDF5 file
hs1_counts <- Read10X_h5('vdj_v1_hs_pbmc3_5gex_filtered_gene_bc_matrices_h5.h5')

# Extract the Gene Expression matrix
gene_expression <- hs1_counts[["Gene Expression"]]

# Create the Seurat object with the Gene Expression modality as the default assay
hs1 <- CreateSeuratObject(counts = gene_expression)

saveRDS(hs1, 'vdj_v1_hs_V1_sc_5gex.rds')


R[write to console]: Error in Read10X_h5("vdj_v1_hs_pbmc3_5gex_filtered_gene_bc_matrices_h5.h5") : 
  File not found




Error in Read10X_h5("vdj_v1_hs_pbmc3_5gex_filtered_gene_bc_matrices_h5.h5") : 
  File not found


RInterpreterError: Failed to parse and evaluate line '\n# Read the data from the HDF5 file\nhs1_counts <- Read10X_h5(\'vdj_v1_hs_pbmc3_5gex_filtered_gene_bc_matrices_h5.h5\')\n\n# Extract the Gene Expression matrix\ngene_expression <- hs1_counts[["Gene Expression"]]\n\n# Create the Seurat object with the Gene Expression modality as the default assay\nhs1 <- CreateSeuratObject(counts = gene_expression)\n\nsaveRDS(hs1, \'vdj_v1_hs_V1_sc_5gex.rds\')\n'.
R error message: 'Error in Read10X_h5("vdj_v1_hs_pbmc3_5gex_filtered_gene_bc_matrices_h5.h5") : \n  File not found'

In [22]:
%%R
#read your Seurat object in
# V1 <- readRDS('conga_example_datasets_v1/vdj_v1_hs_V1_sc_5gex.rds')
V1 <- readRDS('conga_example_datasets_v1/vdj_v1_hs_V1_sc_5gex.rds')

DimPlot(V1) | FeaturePlot( V1, "CD3E")
# Keep in mind, conga will keep only the T cells with paired TCR information

R[write to console]: Error in gzfile(file, "rb") : cannot open the connection

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In gzfile(file, "rb") :
R[write to console]: 
 
R[write to console]:  cannot open compressed file 'conga_example_datasets_v1/vdj_v1_hs_V1_sc_5gex.rds', probable reason 'No such file or directory'




Error in gzfile(file, "rb") : cannot open the connection


RInterpreterError: Failed to parse and evaluate line '#read your Seurat object in\n# V1 <- readRDS(\'conga_example_datasets_v1/vdj_v1_hs_V1_sc_5gex.rds\')\nV1 <- readRDS(\'conga_example_datasets_v1/vdj_v1_hs_V1_sc_5gex.rds\')\n\nDimPlot(V1) | FeaturePlot( V1, "CD3E")\n# Keep in mind, conga will keep only the T cells with paired TCR information\n'.
R error message: 'Error in gzfile(file, "rb") : cannot open the connection'

In [ ]:
%%R -o V1_sce
# R-magics line above tells anndata2ri to convert the dataset to AnnData

#  but first convert your object to SingleCellExperiment Format 
V1_sce <- as.SingleCellExperiment(V1)
V1_sce

In [ ]:
#write to h5ad for now. Ideally be nice to get away from strictly using the path
V1_sce.write("../conga_datasets/processed/vdj_v1_hs_V1_sc_5gex.h5ad")

In [ ]:
%matplotlib inline

#begin conga workflow
import conga
import pandas as pd
import matplotlib.pyplot as plt
from conga.tcrdist.make_10x_clones_file import make_10x_clones_file

In [ ]:
# call the Anndata file and conga on
gex_datafile = "../conga_datasets/processed/vdj_v1_hs_V1_sc_5gex.h5ad"
gex_datatype = 'h5ad' # other possibilities right now: ['10x_mtx', 'h5ad'] (h5ad from scanpy)
tcr_datafile = './testDrive/vdj_v1_hs_pbmc_t_filtered_contig_annotations.csv'
organism = 'human'
clones_file = 'tmp_hs_pbmc_clones.tsv'
outfile_prefix = 'tmp_hs_pbmc'

In [ ]:
# this creates the TCRdist 'clones file'
make_10x_clones_file( tcr_datafile, organism, clones_file )

In [ ]:
# this command will create another file with the kernel PCs for subsequent reading by conga
conga.preprocess.make_tcrdist_kernel_pcs_file_from_clones_file( clones_file, organism )

In [ ]:
adata = conga.preprocess.read_dataset(gex_datafile, gex_datatype, clones_file )

# store the organism info in adata
adata.uns['organism'] = organism

adata

In [ ]:
# top 50 TCRdist kPCS 
adata.obsm['X_pca_tcr'].shape

In [ ]:
# CDR3-alpha regions:
adata.obs['cdr3a'].head(3)

In [ ]:
adata = conga.preprocess.filter_and_scale( adata )

In [ ]:
adata = conga.preprocess.reduce_to_single_cell_per_clone( adata )

adata

In [ ]:
adata = conga.preprocess.cluster_and_tsne_and_umap( adata )

adata

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
xy = adata.obsm['X_gex_2d']
clusters = np.array(adata.obs['clusters_gex'])
cmap = plt.get_cmap('tab20')
colors = [ cmap.colors[x] for x in clusters]
plt.scatter( xy[:,0], xy[:,1], c=colors)
plt.title('GEX UMAP colored by GEX clusters')

plt.subplot(122)
xy = adata.obsm['X_tcr_2d']
clusters = np.array(adata.obs['clusters_tcr'])
cmap = plt.get_cmap('tab20')
colors = [ cmap.colors[x] for x in clusters]
plt.scatter( xy[:,0], xy[:,1], c=colors)
plt.title('TCR UMAP colored by TCR clusters');

In [ ]:
# these are the nbrhood sizes, as a fraction of the entire dataset:
nbr_fracs = [0.01, 0.1]

# we use this nbrhood size for computing the nndists
nbr_frac_for_nndists = 0.01

all_nbrs, nndists_gex, nndists_tcr = conga.preprocess.calc_nbrs(
    adata, nbr_fracs, also_calc_nndists=True, nbr_frac_for_nndists=nbr_frac_for_nndists)

# stash these in obs array, they are used in a few places...                                                                                                                
adata.obs['nndists_gex'] = nndists_gex
adata.obs['nndists_tcr'] = nndists_tcr

conga.preprocess.setup_tcr_cluster_names(adata) #stores in adata.uns

In [ ]:
results = conga.correlations.run_graph_vs_graph(adata, all_nbrs)

conga_scores = adata.obs['conga_scores']

good_mask = ( conga_scores <= 1.0 )
adata.obs['good_score_mask'] = good_mask

print(f'found {np.sum(good_mask)} conga hits')

results.sort_values('conga_score', inplace=True)

results.head(3)

In [ ]:
# write the results to a tsv file
clusters_gex = np.array(adata.obs['clusters_gex'])
clusters_tcr = np.array(adata.obs['clusters_tcr'])

indices = results['clone_index']
results['gex_cluster'] = clusters_gex[indices]
results['tcr_cluster'] = clusters_tcr[indices]
for tag in 'va ja cdr3a vb jb cdr3b'.split():
    results[tag] = list(adata.obs[tag][indices])
tsvfile = outfile_prefix+'_graph_vs_graph_hits.tsv'
print('saving graph-vs-graph results to file:',tsvfile)

results.to_csv(tsvfile, sep='\t', index=False)

In [ ]:
#put the conga hits on top
colors = np.sqrt(np.maximum(-1*np.log10(conga_scores),0.0))
reorder = np.argsort(colors)

plt.figure(figsize=(12,6))
plt.subplot(121)
xy = adata.obsm['X_gex_2d']
plt.scatter( xy[reorder,0], xy[reorder,1], c=colors[reorder], vmin=0, vmax=np.sqrt(5))
plt.title('GEX UMAP colored by conga score')

plt.subplot(122)
xy = adata.obsm['X_tcr_2d']
plt.scatter( xy[reorder,0], xy[reorder,1], c=colors[reorder], vmin=0, vmax=np.sqrt(5))
plt.title('TCR UMAP colored by conga score');

In [ ]:
nbrs_gex, nbrs_tcr = all_nbrs[0.1]

min_cluster_size = 5

# calc tcr sequence features of good cluster pairs
good_bicluster_tcr_scores = conga.correlations.calc_good_cluster_tcr_features(
    adata, good_mask, clusters_gex, clusters_tcr, conga.tcr_scoring.all_tcr_scorenames, verbose=False,
    min_count=min_cluster_size)

# run rank_genes on most common biclusters
rank_genes_uns_tag = 'rank_genes_good_biclusters'
conga.correlations.run_rank_genes_on_good_biclusters(
    adata, good_mask, clusters_gex, clusters_tcr, min_count=min_cluster_size, key_added= rank_genes_uns_tag)

gex_header_tcr_score_names = ['mhci2', 'cdr3len', 'cd8', 'nndists_tcr']

logo_pngfile = outfile_prefix+'_bicluster_logos.png'

conga.plotting.make_logo_plots(
    adata, nbrs_gex, nbrs_tcr, min_cluster_size, logo_pngfile,
    good_bicluster_tcr_scores=good_bicluster_tcr_scores,
    rank_genes_uns_tag = rank_genes_uns_tag,
    gex_header_tcr_score_names = gex_header_tcr_score_names )

In [ ]:
pval_threshold = 1.
results = []
for nbr_frac in nbr_fracs:
    nbrs_gex, nbrs_tcr = all_nbrs[nbr_frac]
    print('finding biased GEX features for nbrhoods with size', nbr_frac, nbrs_gex.shape)
    results.append( conga.correlations.tcr_nbrhood_rank_genes_fast( adata, nbrs_tcr, pval_threshold, verbose=False))
    results[-1]['nbr_frac'] = nbr_frac

# save the results to a file
tsvfile = outfile_prefix+'_tcr_nbr_graph_vs_gex_features.tsv'
print('making:', tsvfile)
results_df = pd.concat(results, ignore_index=True)
results_df.to_csv(tsvfile, index=False, sep='\t')

pngfile = outfile_prefix+'_tcr_nbr_graph_vs_gex_features.png'
print('making:', pngfile)
conga.plotting.plot_ranked_strings_on_cells(
    adata, results_df, 'X_tcr_2d', 'clone_index', 'mwu_pvalue_adj', 1.0, 'feature', pngfile)

In [ ]:
pngfile = outfile_prefix+'_tcr_nbr_graph_vs_gex_features_panels.png'
print('making:', pngfile)
conga.plotting.make_feature_panel_plots(adata, 'tcr', all_nbrs, results_df, pngfile, 'gex')

In [ ]:
pval_threshold = 1.
results = []
tcr_score_names = conga.tcr_scoring.all_tcr_scorenames # the TCR features to use
for nbr_frac in nbr_fracs:
    nbrs_gex, nbrs_tcr = all_nbrs[nbr_frac]
    results.append( conga.correlations.gex_nbrhood_rank_tcr_scores(
        adata, nbrs_gex, tcr_score_names, pval_threshold, verbose=False ))
    results[-1]['nbr_frac'] = nbr_frac
results_df = pd.concat(results, ignore_index=True)

tsvfile = outfile_prefix+'_gex_nbr_graph_vs_tcr_features.tsv'
print('making:', tsvfile)
results_df.to_csv(tsvfile, index=False, sep='\t')

pngfile = outfile_prefix+'_gex_nbr_graph_vs_tcr_features.png'
print('making:', pngfile)

conga.plotting.plot_ranked_strings_on_cells(
    adata, results_df, 'X_gex_2d', 'clone_index', 'mwu_pvalue_adj', 1.0, 'feature', pngfile,
    direction_column='ttest_stat')

In [ ]:
pngfile = outfile_prefix+'_gex_nbr_graph_vs_tcr_features_panels.png'
print('making:', pngfile)
conga.plotting.make_feature_panel_plots(adata, 'gex', all_nbrs, results_df, pngfile, 'tcr')